In [ ]:
import cifti
import numpy as np
import os
import nibabel as nb
import h5py
import copy

In [16]:
pkg_dir = '/home/shared/2018/visual/HCP7TFIXED/'
out_dir = '/home/shared/2018/visual/hcp_cerebellum/'

orig = cifti.read(os.path.join(pkg_dir,'999999','tfMRI_RETEXP_7T_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii'))
series = orig[1][0] # array from 0-300
bm = orig[1][1] # cifti brain model

wbc = """/home/vanes/Downloads/workbench/bin_rh_linux64/wb_command -cifti-separate {cii} \
COLUMN -volume-all {cii_n}_data_sub.nii \
-metric CORTEX_LEFT {cii_n}_L.gii \
-metric CORTEX_RIGHT {cii_n}_R.gii &"""

In [17]:
# read prf results for all subjects:
with h5py.File(os.path.join(pkg_dir,'prfresults.mat'), 'r') as mat:
    allresults = mat['/allresults'].value
    
# this is a huge numpy array now, with dimensions:
# 0: run half
#        0 = all data
#        1 = first half of runs
#        2 = second half of runs
# 1: subjects (len = 184)
# 2: prf param (len = 6)
#        0 = polar angle (in degrees from 0-360)
#        1 = ecc
#        2 = size
#        3 = gain
#        4 = r2
#        5 = mean signal
# 3: voxels (len = 91282)

ciftidims = {
    'polar':0,
    'ecc':1,
    'size':2,
    'gain':3,
    'r2':4,
    'mean':5
}

In [18]:
def compute_in_stim_ratio(xs,ys,sigmas):
    """
    computes the ratio of pRF that falls within the stimulus
    aperture
    """
    
    scope = 4 # times the stimated region
    res = 100 # 4*stim_radius * 
    stim_radius = 8
    n = 0.05
    
    from popeye.spinach import generate_og_receptive_fields

    # define visual space in coordinates
    deg_x, deg_y                    =   np.meshgrid(np.linspace(-stim_radius*scope, stim_radius*scope, res), np.linspace(-stim_radius*scope, stim_radius*scope, res)) 

    # create the rfs using the popeye gaussian function
    rfs                             =   generate_og_receptive_fields(xs.astype(float),ys.astype(float),sigmas.astype(float),np.ones(len(xs)).astype(float),deg_x.astype(float),deg_y.astype(float))                                    
    # apply the non linearity
    css_rfs                               =   rfs ** n
    # compute the total pRF content
    total_prf_content               =   css_rfs.reshape((res**2,-1)).sum(axis=0) 
    # and the amount of the pRF inside the stim region
    stim_vignet                        =   np.sqrt(deg_x ** 2 + deg_y**2) < stim_radius                            
    in_stim_ratio                      =   css_rfs[stim_vignet,:].sum(axis=0) / total_prf_content                 
    in_stim_ratio[np.isnan(in_stim_ratio)] = 0

    return in_stim_ratio, css_rfs, stim_vignet

In [34]:
def gmm_threshold(data,n_components=2,maxrange=100):

    from sklearn.mixture import GMM

    # fit gaussian mixture model to define r2 threshold
    gmm = GMM(n_components = n_components)
    gmm = gmm.fit(np.expand_dims(data,1))

    x = np.linspace(0,maxrange,10000)
    logprob, responsibilities = gmm.score_samples(np.expand_dims(x,1))

    pdf = np.exp(logprob)
    pdf_individual = responsibilities * pdf[:, np.newaxis]

    thresh = x[np.where(pdf_individual[:,0]>pdf_individual[:,1])[0]][0]
    if thresh == 0:
        thresh = x[np.where(pdf_individual[:,1]>pdf_individual[:,0])[0]][0]

    return thresh

# create weighted average subject

The HCP dataset contains an average subject, where the timecourses are averaged, to which pRF models are then fit. This seems like an extremely ugly way to average data across subjects. Potentially much nicer is to create a weighted average subject, weighted on R2 of the pRF model.

In [19]:
# to create average polar angles, we first have to convert to complex numbers:
# convert the angle to radians to complex numbers, and scale r2 values to [0,1]
rads = 2*np.pi*allresults[:,:,ciftidims['polar'],:]/360
real, imag = np.cos(rads) , np.sin(rads) # this ensures that 0 is right
newresults = np.array([real, imag, allresults[:,:,0,:], allresults[:,:,1,:], allresults[:,:,2,:], allresults[:,:,3,:], allresults[:,:,4,:], allresults[:,:,5,:]]).transpose((1,2,0,3))
newresults_dims = {
    'x': 0,
    'y': 1,
    'polar':2,
    'ecc':3,
    'size':4,
    'gain':5,
    'r2':6,
    'mean':7
}

In [37]:
# remove voxels from the 'noise' pool in the averaging procedure:
weights = copy.copy(newresults[:,:,newresults_dims['r2'],:])
weights[weights<2.2] = 0.00001
# weights[weights>=2.2] = 100

# re-code the data for weighted average subject
comp_angle = real + imag * 1j
# scale the complex number by the rsq (i.e. applying weight)
scaled_comp_angle = comp_angle * weights

# now average over subjects, and convert back to radians
avg_angle = np.angle(np.mean(scaled_comp_angle, axis=1))[0]
# avg_angle = avg_angle / (2*np.pi) * 360
# now rescale so it runs from 0-360 again, where 0 is right 
avg_angle[avg_angle <0] = np.pi*2+avg_angle[avg_angle <0]
avg_angle = np.degrees(avg_angle)

# average rsq, ecc and size over subjects:
# rsq_weights = (weights>=2.2).astype(int)
# rsq_weights[rsq_weights==0] = 0.00001

# avg_rsq = np.average(weights,weights=rsq_weights, axis=1)[0]

avg_rsq = np.average(newresults[:,:,newresults_dims['r2'],:],weights= weights, axis=1)[0]

r2thresh = gmm_threshold(np.ravel(avg_rsq))
print 'r2thresh for weighted avg subject:%.3f'%r2thresh
    
avg_ecc = np.average(newresults[:,:,newresults_dims['ecc'],:], axis=1, weights=weights)[0]
avg_size = np.average(newresults[:,:,newresults_dims['size'],:], axis=1, weights=weights)[0]
avg_gain = np.average(newresults[:,:,newresults_dims['gain'],:], axis=1, weights=weights)[0]
avg_meanvol = np.average(newresults[:,:,newresults_dims['mean'],:], axis=1, weights=weights)[0]


# avg_ecc = np.median(newresults[:,:,3,:], axis=1)[0]
# avg_size = np.median(newresults[:,:,7,:], axis=1)[0]

# r2thresh = gmm_threshold(np.ravel(avg_rsq))

# eccthresh = gmm_threshold(np.ravel(avg_ecc))

# sizethresh = gmm_threshold(np.ravel(avg_size))


# print r2thresh, eccthresh, sizethresh
# add mask
# mask = (avg_rsq > 0.4) # pRFs with too poor r2
# mask *= (avg_size > 2) # pRFs larger than 2 degrees
# mask *= (avg_ecc > 0.5) # pRFs closer than 0.5 degrees to fixation

# apply mask
# avg_rsq[~mask] = np.nan
# avg_ecc[~mask] = np.nan
# avg_size[~mask] = np.nan
# avg_angle[~mask] = np.nan

# add as 184th subject
cii = os.path.join(out_dir,'all_subjects','prfresults_subject_184.dscalar.nii')
# print cii
cifti.write(cii, [avg_angle, avg_ecc, avg_gain, avg_meanvol,avg_rsq,avg_size], (cifti.Scalar.from_names(['ang', 'ecc', 'gain', 'meanvol','r2','rfsize']), bm))
# and convert the created cifti file to gii files
wbc_c = wbc.format(cii=cii, cii_n=cii[:-4])
os.system(wbc_c);

/home/vanes/anaconda2/envs/analysis/lib/python2.7/site-packages/ipykernel/__main__.py:15: RuntimeWarning: invalid value encountered in less


r2thresh for weighted avg subject:12.971


# now save ciftis niftis and giftis of all subjects

In [ ]:
# For all subjects
for sj in [45,149]:#range(184):
    
    # get results for this subject
    these_results = copy.copy(allresults[0,sj])
    these_results1 = copy.copy(allresults[1,sj])
    these_results2 = copy.copy(allresults[2,sj])

    # compute euclidean distance between prf centers between run halfs
    xs1 = np.cos(np.radians(np.ravel(these_results_1[ciftidims['polar']]))) * np.ravel(these_results_1[ciftidims['ecc']])
    ys1 = np.sin(np.radians(np.ravel(these_results_1[ciftidims['polar']]))) * np.ravel(these_results_1[ciftidims['ecc']])

    xs2 = np.cos(np.radians(np.ravel(these_results_2[ciftidims['polar']]))) * np.ravel(these_results_2[ciftidims['ecc']])
    ys2 = np.sin(np.radians(np.ravel(these_results_2[ciftidims['polar']]))) * np.ravel(these_results_2[ciftidims['ecc']])

    diff_vectors = np.array([np.ravel([xs1-xs2]),np.ravel([ys1-ys2])]) 
    diff_norms = np.linalg.norm(diff_vectors,axis=0)

    new_results = np.concatenate([allresults[0,sj],diff_norms[np.newaxis,:]],axis=0)
    
    # save results
    cii = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar.nii'%sj)
    cifti.write(cii, new_results, (cifti.Scalar.from_names(['ang', 'ecc', 'gain', 'meanvol', 'R2', 'rfsize','dist']), bm))
    # and convert the created cifti file to gii files
    wbc_c = wbc.format(cii=cii, cii_n=cii[:-4])
    os.system(wbc_c)    


# add in stim ratio to niftis

In [ ]:
dims = {
    'polar':0,
    'ecc':1,
    'r2':4,
    'size':5
}

# Add in stim ratio to niftis
for sj in [45,149]:#range(184):

    # load the prf results nifti
    fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)
    img = nb.load(fn)
    data = img.get_data()

    # compute in stim ratio
    xs = np.cos(np.radians(np.ravel(data[:,:,:,dims['polar']]))) * np.ravel(data[:,:,:,dims['ecc']])
    ys = np.sin(np.radians(np.ravel(data[:,:,:,dims['polar']]))) * np.ravel(data[:,:,:,dims['ecc']])
    
    sizes = data[:,:,:,dims['size']]
    v = (np.ravel(sizes)>0)

    in_stim_ratio = compute_in_stim_ratio(xs[v],ys[v],np.ravel(sizes)[v])[0]
    all_in_stim_ratios = np.zeros(np.shape(sizes))
    all_v = (sizes>0)
    all_in_stim_ratios[all_v] = in_stim_ratio
    
    data = np.concatenate([data,all_in_stim_ratios[:,:,:,np.newaxis]],axis=3)

    # save data (overwrite previous nifti, as its not changing the other dimensions)
    out_fn = os.path.join(out_dir,'all_subjects','prfresults_subject_%d.dscalar_data_sub.nii'%sj)        
    new_data = nb.Nifti1Image(data,affine=img.affine,header=img.header)
    nb.save(new_data,out_fn)
        
    
    